#Data Initialization for Reed Solomon


In [75]:
import random
import copy
import math
random.seed(10)
Data=[]
for i in range(10000):
   Data.append(random.randint(0,7))

print(Data)
print(len(Data))

[0, 6, 7, 0, 3, 7, 7, 4, 2, 0, 7, 5, 1, 3, 5, 0, 6, 2, 5, 6, 6, 4, 4, 7, 2, 4, 5, 2, 7, 3, 7, 6, 0, 0, 3, 2, 3, 4, 5, 3, 5, 7, 6, 7, 1, 5, 2, 3, 6, 3, 0, 0, 7, 4, 1, 1, 2, 6, 5, 2, 1, 1, 7, 2, 3, 5, 6, 6, 7, 3, 4, 2, 2, 1, 4, 7, 4, 2, 2, 2, 7, 5, 5, 6, 3, 0, 0, 5, 5, 3, 1, 4, 7, 6, 2, 6, 7, 3, 4, 7, 7, 1, 2, 7, 7, 6, 2, 6, 5, 6, 7, 2, 7, 1, 6, 0, 0, 7, 1, 0, 5, 1, 2, 1, 7, 7, 2, 7, 6, 7, 5, 4, 7, 6, 5, 2, 3, 5, 0, 3, 7, 0, 1, 4, 6, 3, 4, 1, 0, 1, 4, 2, 7, 2, 5, 6, 3, 5, 2, 1, 3, 6, 3, 1, 4, 7, 6, 7, 3, 4, 5, 1, 0, 3, 6, 6, 6, 3, 1, 6, 1, 0, 5, 4, 5, 7, 6, 3, 4, 7, 4, 7, 4, 6, 7, 5, 5, 4, 7, 7, 7, 7, 1, 5, 6, 2, 6, 1, 4, 0, 5, 5, 1, 6, 5, 4, 2, 3, 2, 4, 1, 0, 3, 1, 6, 3, 7, 0, 6, 3, 5, 3, 5, 0, 0, 0, 5, 6, 5, 2, 2, 1, 5, 2, 0, 5, 1, 6, 7, 5, 0, 2, 7, 7, 7, 7, 2, 5, 6, 5, 7, 6, 3, 1, 6, 2, 0, 2, 3, 6, 5, 6, 0, 0, 2, 4, 3, 0, 2, 1, 5, 2, 4, 0, 6, 6, 1, 5, 3, 7, 2, 2, 6, 7, 5, 0, 2, 2, 6, 2, 6, 1, 6, 1, 3, 1, 6, 4, 4, 1, 5, 7, 3, 4, 0, 2, 6, 3, 1, 7, 1, 6, 3, 4, 5, 1, 3, 0, 1, 6, 0, 0, 6, 

In [69]:
def match_(arr1, arr2):
  length= min(len(arr1),len(arr2))
  for i in range(length):
    if arr1[i] != arr2[i]:
      return False
  return True

#Reed Solomon

In [3]:
!pip install reedsolo==1.6.1   # 1.6.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.3/280.3 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for reedsolo: filename=reedsolo-1.6.1-cp310-cp310-linux_x86_64.whl size=979126 sha256=93f0719f7350915dafad74bbd8d623ea49f02a1d77d4984a6c5572376690cb66
  Stored in directory: /root/.cache/pip/wheels/b6/c0/85/c69301d9cd2ce2947a56f93b18f6815b481bcc60f3aff8c634
Successfully built reedsolo


In [76]:
import reedsolo as rs


class DigitRecover:
    def __init__(self, data_blck_size, rs_blck_size, data_arr_size, rs_arr_size, top_k):
        assert data_blck_size + rs_blck_size < 2 ** 3
        self.rs = rs.RSCodec(nsize=data_blck_size + rs_blck_size, nsym=rs_blck_size, c_exp=3)
        self.data_blck_size = data_blck_size
        self.rs_blck_size = rs_blck_size
        self.data_arr_size = data_arr_size
        self.rs_arr_size = rs_arr_size
        self.top_k = top_k

        self.call = 0


    def calculate(self, digits):
        rscode = self.rs.encode(bytes(digits))
        #print(rscode,"---")
        return list(rscode[self.data_blck_size:])

    def verify(self, digits):
            try:
                return list(self.rs.decode(bytes(digits[0]))[0]), digits[0][self.data_blck_size:], True
            except:
                 return digits[0][:self.data_blck_size], digits[0][self.data_blck_size:], False



    def encode(self, digits):
        rem = len(digits) % self.data_blck_size
        if rem:
            digits = digits + [0] * (self.data_blck_size - rem)
        ret = []
        for i in range(0, len(digits), self.data_blck_size):
            d = digits[i:i + self.data_blck_size]
            e = self.calculate(d)
            ret += e
        return ret

    def combine(self, digits):
        data_val = [digit[:-self.rs_arr_size] for digit in digits]
        csum_val = [digit[-self.rs_arr_size:] for digit in digits]

        rem = len(data_val[0]) % self.data_blck_size
        if rem:
            for r in range(self.top_k):
              data_val[r] = data_val[r] + [0] * (self.data_blck_size - rem)


        digits_alt = [[] for _ in range(self.top_k)]

        i = 0
        j = 0
        while i < len(data_val[0]):
            d = [dv[i:i + self.data_blck_size] for dv in data_val]

            c = [cv[j:j + self.rs_blck_size] for cv in csum_val]

            for k in range(self.top_k):
                digits_alt[k] += d[k] + c[k]

            i += self.data_blck_size
            j += self.rs_blck_size
        return digits_alt

    def recover(self, digits):
        digits_alt = self.combine(digits)
        dat = []
        csum = []
        batch_size = self.data_blck_size + self.rs_blck_size
        cnt1 = 0
        cnt2 = 0
        totalCorrectBlock = 0
        for i in range(0, len(digits[0]), batch_size):
            d = [arr[i:i + batch_size] for arr in digits_alt]
            x, y, b = self.verify(d)
            dat += x
            csum += y
            if b:
                cnt1 += 1
                if(match_(Data[self.call * self.data_blck_size:(self.call + 1) * self.data_blck_size], x) ):
                  totalCorrectBlock += 1

            else:
                cnt2 += 1

            self.call += 1
        return dat[:self.data_arr_size], csum[:self.rs_arr_size], cnt1,cnt2,totalCorrectBlock

#Initialization of parameters

In [77]:
############################# INITIALIZING PARAMETERS
top_1 = 95
data_blck_size = 3
csum_blck_size=3
DataChunk = len(Data)
checksumChunk= int(math.ceil(DataChunk/data_blck_size))*csum_blck_size
top_k=1
#########################################################

dr = DigitRecover(data_blck_size, csum_blck_size, DataChunk, checksumChunk, top_k)
Checksum = dr.encode(Data)

if(checksumChunk!=len(Checksum)):
  print("ERROR in initializing the Checksum Size")


print(Data)
print('Size of array of  sender for Data: ',DataChunk)

print(Checksum)
print('Size of array of  sender for Checksum: ',len(Checksum))

[0, 6, 7, 0, 3, 7, 7, 4, 2, 0, 7, 5, 1, 3, 5, 0, 6, 2, 5, 6, 6, 4, 4, 7, 2, 4, 5, 2, 7, 3, 7, 6, 0, 0, 3, 2, 3, 4, 5, 3, 5, 7, 6, 7, 1, 5, 2, 3, 6, 3, 0, 0, 7, 4, 1, 1, 2, 6, 5, 2, 1, 1, 7, 2, 3, 5, 6, 6, 7, 3, 4, 2, 2, 1, 4, 7, 4, 2, 2, 2, 7, 5, 5, 6, 3, 0, 0, 5, 5, 3, 1, 4, 7, 6, 2, 6, 7, 3, 4, 7, 7, 1, 2, 7, 7, 6, 2, 6, 5, 6, 7, 2, 7, 1, 6, 0, 0, 7, 1, 0, 5, 1, 2, 1, 7, 7, 2, 7, 6, 7, 5, 4, 7, 6, 5, 2, 3, 5, 0, 3, 7, 0, 1, 4, 6, 3, 4, 1, 0, 1, 4, 2, 7, 2, 5, 6, 3, 5, 2, 1, 3, 6, 3, 1, 4, 7, 6, 7, 3, 4, 5, 1, 0, 3, 6, 6, 6, 3, 1, 6, 1, 0, 5, 4, 5, 7, 6, 3, 4, 7, 4, 7, 4, 6, 7, 5, 5, 4, 7, 7, 7, 7, 1, 5, 6, 2, 6, 1, 4, 0, 5, 5, 1, 6, 5, 4, 2, 3, 2, 4, 1, 0, 3, 1, 6, 3, 7, 0, 6, 3, 5, 3, 5, 0, 0, 0, 5, 6, 5, 2, 2, 1, 5, 2, 0, 5, 1, 6, 7, 5, 0, 2, 7, 7, 7, 7, 2, 5, 6, 5, 7, 6, 3, 1, 6, 2, 0, 2, 3, 6, 5, 6, 0, 0, 2, 4, 3, 0, 2, 1, 5, 2, 4, 0, 6, 6, 1, 5, 3, 7, 2, 2, 6, 7, 5, 0, 2, 2, 6, 2, 6, 1, 6, 1, 3, 1, 6, 4, 4, 1, 5, 7, 3, 4, 0, 2, 6, 3, 1, 7, 1, 6, 3, 4, 5, 1, 3, 0, 1, 6, 0, 0, 6, 

In [78]:
import copy


temp= copy.deepcopy(Data)
Data = Data + Checksum


print(Data)
print('Size of array of  sender for Data: ',len(Data))
print(len(temp))


[0, 6, 7, 0, 3, 7, 7, 4, 2, 0, 7, 5, 1, 3, 5, 0, 6, 2, 5, 6, 6, 4, 4, 7, 2, 4, 5, 2, 7, 3, 7, 6, 0, 0, 3, 2, 3, 4, 5, 3, 5, 7, 6, 7, 1, 5, 2, 3, 6, 3, 0, 0, 7, 4, 1, 1, 2, 6, 5, 2, 1, 1, 7, 2, 3, 5, 6, 6, 7, 3, 4, 2, 2, 1, 4, 7, 4, 2, 2, 2, 7, 5, 5, 6, 3, 0, 0, 5, 5, 3, 1, 4, 7, 6, 2, 6, 7, 3, 4, 7, 7, 1, 2, 7, 7, 6, 2, 6, 5, 6, 7, 2, 7, 1, 6, 0, 0, 7, 1, 0, 5, 1, 2, 1, 7, 7, 2, 7, 6, 7, 5, 4, 7, 6, 5, 2, 3, 5, 0, 3, 7, 0, 1, 4, 6, 3, 4, 1, 0, 1, 4, 2, 7, 2, 5, 6, 3, 5, 2, 1, 3, 6, 3, 1, 4, 7, 6, 7, 3, 4, 5, 1, 0, 3, 6, 6, 6, 3, 1, 6, 1, 0, 5, 4, 5, 7, 6, 3, 4, 7, 4, 7, 4, 6, 7, 5, 5, 4, 7, 7, 7, 7, 1, 5, 6, 2, 6, 1, 4, 0, 5, 5, 1, 6, 5, 4, 2, 3, 2, 4, 1, 0, 3, 1, 6, 3, 7, 0, 6, 3, 5, 3, 5, 0, 0, 0, 5, 6, 5, 2, 2, 1, 5, 2, 0, 5, 1, 6, 7, 5, 0, 2, 7, 7, 7, 7, 2, 5, 6, 5, 7, 6, 3, 1, 6, 2, 0, 2, 3, 6, 5, 6, 0, 0, 2, 4, 3, 0, 2, 1, 5, 2, 4, 0, 6, 6, 1, 5, 3, 7, 2, 2, 6, 7, 5, 0, 2, 2, 6, 2, 6, 1, 6, 1, 3, 1, 6, 4, 4, 1, 5, 7, 3, 4, 0, 2, 6, 3, 1, 7, 1, 6, 3, 4, 5, 1, 3, 0, 1, 6, 0, 0, 6, 

#Initialization of Accuracy


In [79]:

sender_val=copy.deepcopy(Data)

Total_Extracted_Val=[]


for i in range(DataChunk):
    Total_Extracted_Val.append(sender_val[i])

for i in range(DataChunk, DataChunk+checksumChunk,1):
    Total_Extracted_Val.append(sender_val[i])




#For Top-1

In [81]:
import math
error_to_be_occured_data= int(math.floor(DataChunk*(100-top_1)/100))
print("error introduced in data: ",error_to_be_occured_data, "\nData Accuracy Calculated: ",(DataChunk-error_to_be_occured_data)*100/DataChunk)

random.seed(12)
error_Index_1=[]
while(True):
     y = random.randrange(DataChunk)
     if y and y+1 not in error_Index_1:
        error_Index_1.append(y)
        error_Index_1.append(y+1)
     if(len(error_Index_1) == error_to_be_occured_data):
       break

print("Index length:", len(error_Index_1))

for i in range(DataChunk):
  if i in error_Index_1:
    Total_Extracted_Val[i] = (Total_Extracted_Val[i]+1)%8



#For Checksum

error_to_be_occured_chksum= int(math.floor(checksumChunk*(100-top_1)/100))
print("error introduced in Checksum: ",error_to_be_occured_chksum, "\nChecksum Accuracy Calculated: ",(checksumChunk-error_to_be_occured_chksum)*100/checksumChunk)

random.seed(21)
error_Index_1c=[]
while(True):
     y = random.randrange(DataChunk, DataChunk+checksumChunk)
     if y not in error_Index_1c:
        error_Index_1c.append(y)
     if(len(error_Index_1c) == error_to_be_occured_chksum):
       break

print("Index length:", len(error_Index_1c))

for i in range(DataChunk, DataChunk+checksumChunk):
  if i in error_Index_1c:
    Total_Extracted_Val[i] = (Total_Extracted_Val[i]+1)%4



error introduced in data:  500 
Data Accuracy Calculated:  95.0
Index length: 500
error introduced in Checksum:  500 
Checksum Accuracy Calculated:  95.00099980003999
Index length: 500


#Retrived Data

In [83]:

print("Top-1: ",top_1)

print("Total received Data", len(Total_Extracted_Val))


incorrect=0
for i in range(len(sender_val)):
  if(sender_val[i]!=Total_Extracted_Val[i]):
    incorrect+=1

print('Total incorrect before decoding: ',incorrect)
TOP1_accuracy= (len(sender_val)-incorrect)/len(sender_val)
TOP1_ErrorCount = incorrect
print('Top-1 Data Accuracy: ',TOP1_accuracy)


Top-1:  95
Total received Data 20002
Total incorrect before decoding:  999
Top-1 Data Accuracy:  0.95005499450055


#Block Accuracy

In [84]:

correct=0
index = DataChunk
for i in range(0,DataChunk,data_blck_size):
  if(match_(sender_val[i:i+data_blck_size],Total_Extracted_Val[i:i+data_blck_size])):
    correct+=1
  index+=csum_blck_size

total_data_block = int(math.ceil(DataChunk/data_blck_size))
print("Total Data Block: ",total_data_block)
print('Initial correct data block: ',correct)
print("(",data_blck_size+csum_blck_size,")"," block accuracy: ",correct/total_data_block*100)
print('Initial Blockwise correct data: ',correct*data_blck_size)

Total Data Block:  3334
Initial correct data block:  3024
( 6 )  block accuracy:  90.70185962807439
Initial Blockwise correct data:  9072


#Data Extraction

In [85]:
digits=[]
confs=[]

digits.append(Total_Extracted_Val)   # We do not want to use permutation encoding in Reed Solomon


receiver_data, receiver_Checksum,successExtraction, failedExtraction,totalCorrectBlock = dr.recover(digits)
print(len(receiver_data), len(receiver_Checksum))
print("Successfully match checksum : ",successExtraction," times")
print("Failed to match checksum and put top-1 : ",failedExtraction," times")
print("Total Matched Data : ",totalCorrectBlock)
print("Total Blockwise correct data after RS : ",totalCorrectBlock*data_blck_size)


10000 10002
Successfully match checksum :  3112  times
Failed to match checksum and put top-1 :  222  times
Total Matched Data :  3104
Total Blockwise correct data after RS :  9312


#RS Block Accuracy

In [86]:
correct=0

data_blck_size = 4

for i in range(0,DataChunk,data_blck_size):
  if(match_(sender_val[i:i+data_blck_size],receiver_data[i:i+data_blck_size])):
    correct+=1

print('Total correct data block: ',correct)
total_data_block = int(math.ceil(DataChunk/data_blck_size))
print("(",data_blck_size+csum_blck_size,")"," block accuracy: ",correct/total_data_block*100)

Total correct data block:  2247
( 7 )  block accuracy:  89.88000000000001


#Cell Accuracy

In [87]:
incorrect=0
for i in range(DataChunk):
  if(sender_val[i]!=Total_Extracted_Val[i]):
    incorrect+=1

print('Total incorrect BEFORE DECODING (data only): ',incorrect,'\nData Extraction Accuracy: ',(DataChunk-incorrect)/DataChunk)
print("---------------------------------------------------")

DataErrorInitial= incorrect

incorrect=0
for i in range(DataChunk):
  if(sender_val[i]!=receiver_data[i]):
    incorrect+=1

DataErrorafterDecoding= incorrect
print('Total incorrect AFTER DECODING (data only): ',incorrect,'\nData Extraction Accuracy: ',(DataChunk-incorrect)/DataChunk)
print()
totalCorrectData = len(receiver_data)-incorrect
print("Data segment size: ",len(receiver_data))
print("Total correct Data: ",totalCorrectData,"(",totalCorrectData/DataChunk*100,"%)")
DataErrorCorrected = DataErrorInitial - DataErrorafterDecoding

Total incorrect BEFORE DECODING (data only):  499 
Data Extraction Accuracy:  0.9501
---------------------------------------------------
Total incorrect AFTER DECODING (data only):  402 
Data Extraction Accuracy:  0.9598

Data segment size:  10000
Total correct Data:  9598 ( 95.98 %)


#Debugging Statistics


In [32]:
from tensorflow.python.ops.check_ops import assert_greater_equal_v2

count1 = 0 # CASE: 1  : initial checksum and Data both correct
count2 = 0 # CASE: 2  : error in either, but successfully corrected
count3 = 0 # CASE: 2  : error in either, but successfully corrected by returning top-1
count4 = 0 # CASE 3(a): error in either, but NOT corrected successfully (matched checksum with incorrect data)
count5 = 0 # CASE 3(b): error in either, but NOT corrected successfully (could not match within available choices)


array1 = []
array2 = []
array3 = []
array4 = []
array5 = []


total_nonmatched = 0

def counterror(arr1, arr2):
  length= min(len(arr1),len(arr2))
  t = 0
  for i in range(length):
    if arr1[i] != arr2[i]:
      t += 1
  return t

def match_(arr1, arr2):
  length= min(len(arr1),len(arr2))
  for i in range(length):
    if arr1[i] != arr2[i]:
      return False
  return True

batch_size =  data_blck_size # Here we assume data block size 4
total_data_block = int(math.ceil(DataChunk/data_blck_size))

index2=0
#match_(sender_val[i:i+data_blck_size],receiver_data[i:i+data_blck_size]) and match_(sender_val[index:index+csum_blck_size],receiver_Checksum[index2:index2+csum_blck_size])

for i in range(0,DataChunk,data_blck_size):
  if(match_(sender_val[i:i+data_blck_size], Total_Extracted_Val[i:i+data_blck_size]) and match_(sender_val[i:i+data_blck_size], receiver_data[i:i+data_blck_size])):
      count1 += 1 # CASE: 1 --> initial checksum and Data both correct
      array1.append(sender_val[i:i+data_blck_size])
      array1.append(receiver_data[i:i+data_blck_size])
      array1.append(Total_Extracted_Val[i:i+data_blck_size])

  elif(match_(sender_val[i:i+data_blck_size], receiver_data[i:i+data_blck_size])):
      count2 += 1 # CASE: 2 --> error in either, but successfully corrected
      array2.append(sender_val[i:i+data_blck_size])
      array2.append(receiver_data[i:i+data_blck_size])
      array2.append(Total_Extracted_Val[i:i+data_blck_size])



  elif(not match_(sender_val[i:i+data_blck_size], receiver_data[i:i+data_blck_size])):
      count4 += 1 # CASE: 3 -->  error in either, but NOT corrected successfully
                  # (a) error in both data and checksum (matched a checksum with incorrect Data)
      total_nonmatched += counterror(sender_val[i:i+data_blck_size], receiver_data[i:i+data_blck_size])
      array4.append(sender_val[i:i+data_blck_size])
      array4.append(receiver_data[i:i+data_blck_size])
      array4.append(Total_Extracted_Val[i:i+data_blck_size])



print("total Data Block: ",total_data_block)
print(count1, count2, count4)
print()
print("Checksum and Data both correct:                     ",count1,"(","{:.2%}".format(count1/total_data_block*1.0),")" )
print("Error in either, but successfully corrected:         ",count2,"(","{:.2%}".format((count2)/total_data_block*1.0),")" )
print("Error in either, but NOT successfully corrected:     ",count4,"(","{:.2%}".format((count4)/total_data_block*1.0),")" )


print("\n\nTotal Correct block: ",count1+count2,"(","{:.2%}".format((count1+count2)/total_data_block*1.0),")" )

if(count4!=0):
    print("Average error in non successful block: ", total_nonmatched/(count4))


total Data Block:  2500
1643 508 349

Checksum and Data both correct:                      1643 ( 65.72% )
Error in either, but successfully corrected:          508 ( 20.32% )
Error in either, but NOT successfully corrected:      349 ( 13.96% )


Total Correct block:  2151 ( 86.04% )
Average error in non successful block:  1.2063037249283668


In [33]:
count1 = 0 # CASE: 1  : initial checksum and Data both correct
count2 = 0 # CASE: 2  : error in either, but successfully corrected
count4 = 0 # CASE 3(a): error in either, but NOT corrected successfully (matched checksum with incorrect data)

# print("\n\n\nCASE---1:In intial Checksum and Block Data both correct: ",len(array1)/3,"\n\n")
# for i in range(1,len(array1),3):
#   print(array1[i-1],"sender val")
#   print(array1[i],"Decoded val")
#   print(array1[i+1],"Top-1 val")
#   print("*"*20)

# print("\n\n\nCASE---2: error in either, but successfully corrected",len(array2)/3,"\n\n")
# for i in range(1,len(array2),3):
#   print(array2[i-1],"sender val")
#   print(array2[i],"Decoded val")
#   print(array2[i+1],"Top-1 val")
#   print("*"*20)


print("\n\n\n CASE 3(a): error in either, but NOT corrected successfully",len(array4)/3,"\n\n")
for i in range(1,len(array4),3):
  print(array4[i-1],"sender val")
  print(array4[i],"Decoded val")
  print(array4[i+1],"Top-1 val")
  print("*"*20)






 CASE 3(a): error in either, but NOT corrected successfully 349.0 


[1, 1, 7, 2] sender val
[2, 1, 7, 2] Decoded val
[2, 1, 7, 2] Top-1 val
********************
[3, 5, 6, 6] sender val
[3, 6, 6, 6] Decoded val
[3, 6, 6, 6] Top-1 val
********************
[5, 3, 1, 4] sender val
[5, 3, 1, 5] Decoded val
[5, 3, 1, 5] Top-1 val
********************
[5, 1, 2, 1] sender val
[5, 1, 2, 2] Decoded val
[5, 1, 2, 2] Top-1 val
********************
[7, 7, 2, 7] sender val
[7, 7, 3, 7] Decoded val
[7, 7, 3, 7] Top-1 val
********************
[3, 5, 0, 3] sender val
[4, 6, 0, 3] Decoded val
[4, 6, 0, 3] Top-1 val
********************
[3, 6, 3, 1] sender val
[3, 6, 3, 2] Decoded val
[3, 6, 3, 2] Top-1 val
********************
[4, 7, 6, 7] sender val
[5, 7, 6, 7] Decoded val
[5, 7, 6, 7] Top-1 val
********************
[3, 4, 5, 1] sender val
[3, 5, 5, 1] Decoded val
[3, 5, 5, 1] Top-1 val
********************
[4, 0, 5, 5] sender val
[4, 0, 5, 6] Decoded val
[4, 0, 5, 6] Top-1 val
*******************